<a href="https://colab.research.google.com/github/melhem-m/AI-Training-Colab/blob/main/Hydraulic_Plant_Analysis_MM_CS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://archive.ics.uci.edu/dataset/447/condition+monitoring+of+hydraulic+systems

Motor VS Vibration Study

Files to upload VS and EPS

In [ ]:
# 1. Install packages (use your working stack)# Fixed stack for Gradio on Colab (Python 3.12)
# ==========================================================
# STABLE INSTALLATION BLOCK (DO NOT CHANGE)
# ==========================================================

!pip install numpy==1.26.4
!pip install gradio==4.44.0
!pip install websockets==10.4
!pip install uvicorn==0.22.0
!pip install fastapi==0.103.0
!pip install starlette==0.27.0

!pip install scikit-learn scipy matplotlib pandas

import os
os.environ["GRADIO_ALLOW_PLATFORM_CHANGE"] = "false"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-pytho

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 169, in _make_candidate_from_dist
    base = self._installed_candidate_cache[dist.canonical_name]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
KeyError: 'hf-xet'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packa

In [3]:
#2. Imports & data loading (EPS1.txt & VS1.txt)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import skew, kurtosis
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import gradio as gr

plt.rcParams["figure.figsize"] = (8, 4)

DATA_PATH = "/content/condition_monitoring"  # adjust if needed

eps1_path = os.path.join(DATA_PATH, "EPS1.txt")
vs1_path  = os.path.join(DATA_PATH, "VS1.txt")

if not os.path.exists(eps1_path):
    raise FileNotFoundError(f"EPS1.txt not found at {eps1_path}")
if not os.path.exists(vs1_path):
    raise FileNotFoundError(f"VS1.txt not found at {vs1_path}")

# UCI files are whitespace-separated, no header
eps1_raw = pd.read_csv(eps1_path, sep=r"\s+", header=None, engine="python")
vs1_raw  = pd.read_csv(vs1_path,  sep=r"\s+", header=None, engine="python")

print("EPS1_raw shape:", eps1_raw.shape)
print("VS1_raw  shape:", vs1_raw.shape)
eps1_raw.head()


EPS1_raw shape: (2205, 6000)
VS1_raw  shape: (2205, 60)


,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2409.6,...,2409.6,2409.2,2409.6,2409.4,2409.6,2409.4,2409.6,2409.6,2409.6,2409.6
1,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,...,2398.8,2398.2,2398.2,2398.0,2398.0,2398.0,2398.0,2397.8,2397.8,2397.8
2,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2395.8,...,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8
3,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2382.8,2382.8,...,2373.2,2372.8,2372.6,2372.4,2372.2,2372.0,2372.0,2372.0,2372.0,2372.0
4,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2373.0,...,2370.0,2370.0,2369.8,2369.8,2369.8,2369.8,2369.6,2369.6,2369.6,2369.6


In [5]:
#3. Cycle-level features for EPS1 & VS1
def compute_features_for_matrix(mat: pd.DataFrame, prefix: str):
    vals = mat.values.astype(float)

    features = pd.DataFrame({
        "cycle_id": np.arange(1, vals.shape[0] + 1),
        f"{prefix}_mean":  vals.mean(axis=1),
        f"{prefix}_max":   vals.max(axis=1),
        f"{prefix}_min":   vals.min(axis=1),
        f"{prefix}_std":   vals.std(axis=1),

        # FIX: NumPy 2.0 removed ndarray.ptp → must use np.ptp
        f"{prefix}_ptp":   np.ptp(vals, axis=1),

        f"{prefix}_rms":   np.sqrt((vals**2).mean(axis=1)),
        f"{prefix}_energy": (vals**2).sum(axis=1),
        f"{prefix}_skew":  skew(vals, axis=1),
        f"{prefix}_kurt":  kurtosis(vals, axis=1),
    })

    return features

eps1_feat = compute_features_for_matrix(eps1_raw, "EPS1")
vs1_feat  = compute_features_for_matrix(vs1_raw,  "VS1")

cycles = pd.merge(eps1_feat, vs1_feat, on="cycle_id", how="inner")
print("cycles shape:", cycles.shape)
cycles.head()


cycles shape: (2205, 19)


,cycle_id,EPS1_mean,EPS1_max,EPS1_min,EPS1_std,EPS1_ptp,EPS1_rms,EPS1_energy,EPS1_skew,EPS1_kurt,VS1_mean,VS1_max,VS1_min,VS1_std,VS1_ptp,VS1_rms,VS1_energy,VS1_skew,VS1_kurt
0,1,2538.929167,2951.6,2339.8,185.600652,611.8,2545.704012,3.888365e+10,1.239186,0.425555,0.576950,0.624,0.532,0.026852,0.092,0.577575,20.015539,-0.044978,-1.275101
1,2,2531.498900,2947.4,2332.0,189.924260,615.4,2538.613382,3.866735e+10,1.266706,0.423567,0.565850,0.626,0.524,0.027013,0.102,0.566494,19.254955,0.770789,-0.238839
2,3,2519.928000,2939.8,2318.6,190.720954,621.2,2527.135059,3.831847e+10,1.270605,0.425694,0.576533,0.662,0.529,0.036422,0.133,0.577683,20.023034,0.934103,-0.042351
3,4,2511.541633,2929.6,2310.8,191.254667,618.8,2518.813158,3.806652e+10,1.277350,0.441855,0.569267,0.645,0.527,0.033184,0.118,0.570233,19.509942,1.007775,-0.165555
4,5,2503.449500,2923.8,2301.8,191.242430,622.0,2510.743529,3.782300e+10,1.280943,0.454105,0.577367,0.660,0.524,0.033203,0.136,0.578321,20.067284,0.795914,0.027945


In [6]:
#4. Data-driven thresholds (from actual values)
eps1_thresh_default = float(np.percentile(cycles["EPS1_mean"], 90))
vs1_thresh_default  = float(np.percentile(cycles["VS1_mean"],  90))

print("Suggested initial thresholds (90th percentile):")
print("  EPS1_mean <", eps1_thresh_default)
print("  VS1_mean  >", vs1_thresh_default)


Suggested initial thresholds (90th percentile):
  EPS1_mean < 2599.022640000062
  VS1_mean  > 0.7149666666666669


5. Risk flag + capsules (events)

Risk = 1 if EPS1_mean < eps_threshold AND VS1_mean > vs_threshold

In [7]:
def compute_risk_and_capsules(eps_thr: float, vs_thr: float):
    df = cycles.copy()

    df["risk"] = (
        (df["EPS1_mean"] < eps_thr) &
        (df["VS1_mean"]  > vs_thr)
    ).astype(int)

    # Build capsules: consecutive cycles with risk == 1
    events = []
    in_event = False
    start = None

    for i, row in df.iterrows():
        if row["risk"] == 1 and not in_event:
            in_event = True
            start = row["cycle_id"]
        elif row["risk"] == 0 and in_event:
            end = df.loc[i-1, "cycle_id"]
            segment = df[(df["cycle_id"] >= start) & (df["cycle_id"] <= end)]
            events.append({
                "event_id": len(events) + 1,
                "start_cycle": int(start),
                "end_cycle":   int(end),
                "duration_cycles": int(end - start + 1),
                "max_EPS1_mean": float(segment["EPS1_mean"].max()),
                "max_VS1_mean":  float(segment["VS1_mean"].max()),
            })
            in_event = False
            start = None

    # If ended in an event
    if in_event and start is not None:
        end = df["cycle_id"].iloc[-1]
        segment = df[(df["cycle_id"] >= start) & (df["cycle_id"] <= end)]
        events.append({
            "event_id": len(events) + 1,
            "start_cycle": int(start),
            "end_cycle":   int(end),
            "duration_cycles": int(end - start + 1),
            "max_EPS1_mean": float(segment["EPS1_mean"].max()),
            "max_VS1_mean":  float(segment["VS1_mean"].max()),
        })

    events_df = pd.DataFrame(events)
    return df, events_df

# test with default thresholds
cycles_with_risk, events_default = compute_risk_and_capsules(
    eps1_thresh_default, vs1_thresh_default
)
print("Number of risk events:", len(events_default))
events_default.head()


Number of risk events: 17


,event_id,start_cycle,end_cycle,duration_cycles,max_EPS1_mean,max_VS1_mean
0,1,213,213,1,2425.901633,0.806367
1,2,215,216,2,2570.476500,0.743633
2,3,249,249,1,2419.997700,0.748733
3,4,291,291,1,2402.880933,0.725300
4,5,366,366,1,2553.137367,0.728033


In [8]:
# 6. Simple decision tree: when is risk = 1?
# Features for the tree
X = cycles[["EPS1_mean", "EPS1_std", "VS1_mean", "VS1_std"]].values
y = cycles_with_risk["risk"].values  # 0 or 1 based on default thresholds

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))

def plot_decision_tree():
    fig, ax = plt.subplots(figsize=(10, 6))
    plot_tree(
        tree,
        feature_names=["EPS1_mean", "EPS1_std", "VS1_mean", "VS1_std"],
        class_names=["No Risk", "Risk"],
        filled=True,
        ax=ax
    )
    fig.tight_layout()
    return fig



              precision    recall  f1-score   support

           0       0.99      1.00      0.99       545
           1       0.67      0.29      0.40         7

    accuracy                           0.99       552
   macro avg       0.83      0.64      0.70       552
weighted avg       0.99      0.99      0.99       552



In [9]:
# 7. Plot helpers (trend, scatter, capsules)

max_cycle_id = int(cycles["cycle_id"].max())

def plot_trends(eps_thr, vs_thr, start_cycle, end_cycle):
    df, _ = compute_risk_and_capsules(eps_thr, vs_thr)
    df = df[(df["cycle_id"] >= start_cycle) & (df["cycle_id"] <= end_cycle)]

    fig, ax1 = plt.subplots(figsize=(10, 4))

    ax1.plot(df["cycle_id"], df["EPS1_mean"], label="EPS1_mean (motor power)")
    ax1.axhline(eps_thr, color="red", linestyle="--", label="EPS1 threshold")
    ax1.set_ylabel("EPS1_mean")

    ax2 = ax1.twinx()
    ax2.plot(df["cycle_id"], df["VS1_mean"], color="green", label="VS1_mean (vibration)")
    ax2.axhline(vs_thr, color="orange", linestyle="--", label="VS1 threshold")
    ax2.set_ylabel("VS1_mean")

    ax1.set_xlabel("Cycle")
    ax1.set_title("Motor Power (EPS1) & Vibration (VS1) over cycles")

    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left", fontsize=8)

    fig.tight_layout()
    return fig


def plot_scatter(eps_thr, vs_thr):
    df, _ = compute_risk_and_capsules(eps_thr, vs_thr)

    fig, ax = plt.subplots(figsize=(6, 6))
    risk_mask = df["risk"] == 1

    ax.scatter(df.loc[~risk_mask, "EPS1_mean"],
               df.loc[~risk_mask, "VS1_mean"],
               alpha=0.5, label="No Risk")

    ax.scatter(df.loc[risk_mask, "EPS1_mean"],
               df.loc[risk_mask, "VS1_mean"],
               alpha=0.8, label="Risk", color="red")

    ax.axvline(eps_thr, color="red", linestyle="--")
    ax.axhline(vs_thr,  color="orange", linestyle="--")

    ax.set_xlabel("EPS1_mean (motor power)")
    ax.set_ylabel("VS1_mean (vibration)")
    ax.set_title("Risk Map: EPS1 vs VS1")
    ax.legend()
    fig.tight_layout()
    return fig


def summarize_events(eps_thr, vs_thr):
    _, events_df = compute_risk_and_capsules(eps_thr, vs_thr)
    if events_df.empty:
        return "No high-risk capsules detected for the current thresholds."
    else:
        summary = []
        summary.append(f"Detected {len(events_df)} high-risk event(s).")
        summary.append("")
        for _, row in events_df.iterrows():
            summary.append(
                f"Event {int(row['event_id'])}: cycles "
                f"{int(row['start_cycle'])}–{int(row['end_cycle'])} "
                f"(duration {int(row['duration_cycles'])}), "
                f"max EPS1_mean={row['max_EPS1_mean']:.2f}, "
                f"max VS1_mean={row['max_VS1_mean']:.3f}"
            )
        return "\n".join(summary)


In [10]:
# 8. Simple chatbot logic
def chatbot_fn(message, history, eps_thr, vs_thr):
    msg_lower = message.lower()
    df, events_df = compute_risk_and_capsules(eps_thr, vs_thr)

    if "threshold" in msg_lower:
        reply = (
            f"Current thresholds:\n"
            f"- EPS1_mean < {eps_thr:.2f}\n"
            f"- VS1_mean  > {vs_thr:.3f}"
        )

    elif "event" in msg_lower or "capsule" in msg_lower:
        if events_df.empty:
            reply = "No high-risk events (capsules) found with the current thresholds."
        else:
            reply = summarize_events(eps_thr, vs_thr)

    elif "risk" in msg_lower or "high vibration" in msg_lower or "high power" in msg_lower:
        n_risk_cycles = int(df["risk"].sum())
        reply = (
            f"There are {n_risk_cycles} cycles classified as high risk "
            f"(EPS1_mean < {eps_thr:.2f} AND VS1_mean > {vs_thr:.3f})."
        )
        if not events_df.empty:
            reply += "\n\n" + summarize_events(eps_thr, vs_thr)

    elif "tree" in msg_lower or "rule" in msg_lower:
        reply = (
            "The decision tree was trained to classify 'Risk' based on "
            "EPS1_mean, EPS1_std, VS1_mean, and VS1_std.\n"
            "You can see the structure in the 'Decision Tree' tab."
        )

    else:
        reply = (
            "I can help you with:\n"
            "- Current thresholds\n"
            "- Number of risk cycles\n"
            "- Event (capsule) summaries\n"
            "- Where high power & high vibration occur\n\n"
            "Try asking: 'How many risk events are there?' or "
            "'Show me the thresholds'."
        )

    return reply


In [11]:
import matplotlib.pyplot as plt

def plot_capsules(eps_thr, vs_thr):
    df, events_df = compute_risk_and_capsules(eps_thr, vs_thr)
    fig, ax = plt.subplots(figsize=(10, 2))

    ax.set_title("High-risk Capsules over Cycles")
    ax.set_xlabel("Cycle")
    ax.set_yticks([])

    if events_df is None or events_df.empty:
        ax.text(0.5, 0.5, "No high-risk capsules for current thresholds.",
                ha="center", va="center", transform=ax.transAxes)
        fig.tight_layout()
        return fig

    # Draw shaded spans for events
    for _, row in events_df.iterrows():
        start = row["start_cycle"]
        end   = row["end_cycle"]
        ax.axvspan(start, end, alpha=0.3)
        ax.text((start + end) / 2, 0.5, f"E{int(row['event_id'])}",
                ha="center", va="center", fontsize=8,
                transform=ax.get_xaxis_transform())

    ax.set_xlim(1, max_cycle_id)
    fig.tight_layout()
    return fig


CHAT BOT 1 with Trends - Option 1 Need more work on the Inputs and Capsules

In [13]:
# ============================================================
# STEP 9 – FULL VERSION (Dashboard + Chat + Explanation Mode)
# ============================================================

import re
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr


# ============================================================
# Helper: Clamp cycle range
# ============================================================
def clamp_range(start, end):
    start = int(start)
    end = int(end)
    if start > end:
        start, end = end, start
    start = max(1, start)
    end = min(max_cycle_id, end)
    return start, end


# ============================================================
# Capsule Plot (fixed)
# ============================================================
def plot_capsules(eps_thr, vs_thr, start_cycle=None, end_cycle=None):
    df, events_df = compute_risk_and_capsules(eps_thr, vs_thr)

    start_cycle = start_cycle or 1
    end_cycle = end_cycle or max_cycle_id
    start_cycle, end_cycle = clamp_range(start_cycle, end_cycle)

    if events_df is not None and not events_df.empty:
        events_df = events_df[
            (events_df["end_cycle"] >= start_cycle) &
            (events_df["start_cycle"] <= end_cycle)
        ]

    fig, ax = plt.subplots(figsize=(10, 2))
    ax.set_title(f"High-risk Capsules ({start_cycle}–{end_cycle})")
    ax.set_xlabel("Cycle")
    ax.set_yticks([])

    if events_df is None or events_df.empty:
        ax.text(0.5, 0.5, "No capsules in this range.",
                ha="center", va="center", transform=ax.transAxes)
        return fig

    for _, row in events_df.iterrows():
        s = max(row["start_cycle"], start_cycle)
        e = min(row["end_cycle"], end_cycle)
        ax.axvspan(s, e, alpha=0.3)
        ax.text((s + e) / 2, 0.5, f"E{int(row['event_id'])}",
                ha="center", va="center", fontsize=8,
                transform=ax.get_xaxis_transform())

    ax.set_xlim(start_cycle, end_cycle)
    return fig


# ============================================================
# Help Figure
# ============================================================
def plot_help_figure():
    fig, ax = plt.subplots(figsize=(8, 4))
    txt = (
        "You can ask:\n"
        " - show trend 200 to 500\n"
        " - scatter 200 to 500\n"
        " - show capsules 800 to 1000\n"
        " - summarize events\n"
        " - explain spike around 1000\n"
        " - find anomalies\n"
    )
    ax.text(0.02, 0.98, txt, ha="left", va="top")
    ax.axis("off")
    return fig


# ============================================================
# Cycle + threshold parser
# ============================================================
def parse_request_params(msg, state, slider_eps_thr, slider_vs_thr):
    nums = [float(n) for n in re.findall(r"\d+\.?\d*", msg)]
    eps_thr_used = slider_eps_thr
    vs_thr_used = slider_vs_thr
    source = "slider"

    if len(nums) >= 4 and ("over" in msg or ">" in msg):
        start_cycle, end_cycle = int(nums[0]), int(nums[1])
        eps_thr_used, vs_thr_used = nums[2], nums[3]
        source = "query"
    else:
        if len(nums) >= 2:
            start_cycle, end_cycle = int(nums[0]), int(nums[1])
        elif len(nums) == 1:
            start_cycle = end_cycle = int(nums[0])
        else:
            start_cycle = state.get("last_start", max(1, max_cycle_id - 199))
            end_cycle = state.get("last_end", max_cycle_id)

    start_cycle, end_cycle = clamp_range(start_cycle, end_cycle)
    return start_cycle, end_cycle, eps_thr_used, vs_thr_used, source


# ============================================================
# Explanation Mode: zoom window + stats + interpretation
# ============================================================
def plot_explanation(df, cycle_center, window=150):
    start = max(1, cycle_center - window)
    end = min(max_cycle_id, cycle_center + window)

    df_win = df[(df["cycle_id"] >= start) & (df["cycle_id"] <= end)]

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(df_win["cycle_id"], df_win["VS1_mean"], label="VS1_mean", color="red")
    ax.plot(df_win["cycle_id"], df_win["EPS1_mean"], label="EPS1_mean", color="blue")
    ax.axvline(cycle_center, color="black", linestyle="--")
    ax.set_title(f"Zoomed analysis around cycle {cycle_center}")
    ax.legend()
    fig.tight_layout()

    stats = {
        "vs1_mean": float(df_win["VS1_mean"].mean()),
        "vs1_max": float(df_win["VS1_mean"].max()),
        "eps_mean": float(df_win["EPS1_mean"].mean()),
        "eps_min": float(df_win["EPS1_mean"].min())
    }

    return fig, stats


# ============================================================
# Intent Detection (Advanced NLP)
# ============================================================
def detect_intent(msg):
    msg = msg.lower()

    # Explanation / diagnostic mode
    if any(k in msg for k in [
        "explain", "cause", "caused", "why", "reason", "what happened",
        "spike", "zone", "high vibration", "vibration zone"
    ]):
        return "explain"

    if any(k in msg for k in ["scatter", "correlation", "relationship", "anomaly"]):
        return "scatter"

    if any(k in msg for k in ["capsule", "interval", "event", "burst"]):
        return "capsules"

    if any(k in msg for k in ["failure", "precursor", "unstable", "transition"]):
        return "failure"

    if "threshold" in msg:
        return "thresholds"

    if any(k in msg for k in ["trend", "plot", "show", "line", "visualize"]):
        return "trend"

    return None


# ============================================================
# CHATBOT HANDLER
# ============================================================
def chat_handle(message, state, slider_eps_thr, slider_vs_thr):
    if state is None:
        state = {"history": [], "last_start": 1, "last_end": max_cycle_id}

    history = state["history"]
    msg_raw = message or ""
    msg = msg_raw.lower().strip()

    history.append({"role": "user", "content": msg_raw})

    if not msg:
        reply = "Try: 'Explain the spike around cycle 1000'"
        history.append({"role": "assistant", "content": reply})
        return history, state, "", plot_help_figure(), ""

    # Detect mode
    mode = detect_intent(msg)

    # Parse cycles
    start_cycle, end_cycle, eps_thr_used, vs_thr_used, src = \
        parse_request_params(msg, state, slider_eps_thr, slider_vs_thr)

    # Compute risk table
    df, events_df = compute_risk_and_capsules(eps_thr_used, vs_thr_used)

    # ============================================================
    # EXPLANATION MODE
    # ============================================================
    if mode == "explain":
        nums = re.findall(r"\d+", msg)
        c = int(nums[0]) if nums else (start_cycle + end_cycle) // 2

        fig, stats = plot_explanation(df, c)

        reply = (
            f"### Explanation for behavior near cycle {c}\n"
            f"- Local VS1 mean: **{stats['vs1_mean']:.3f}**\n"
            f"- Local VS1 max: **{stats['vs1_max']:.3f}**\n"
            f"- Local EPS1 mean: **{stats['eps_mean']:.2f}**\n"
            f"- Local EPS1 min: **{stats['eps_min']:.2f}**\n\n"
            f"### Interpretation:\n"
            f"- Vibration increased sharply while motor load stayed low or stable.\n"
            f"- This pattern indicates **mechanical friction**, **pump wear**, or **incipient cavitation**.\n"
            f"- VS1 increase without EPS rise = system struggling mechanically.\n\n"
            f"### Recommendation:\n"
            f"- Inspect rotating parts / bearings.\n"
            f"- Check hydraulic fluid for aeration.\n"
            f"- Review load conditions around cycle {c}."
        )

        history.append({"role": "assistant", "content": reply})
        return history, state, "", fig, ""

    # ============================================================
    # TREND MODE
    # ============================================================
    if mode == "trend":
        fig = plot_trends(eps_thr_used, vs_thr_used, start_cycle, end_cycle)
        reply = (
            f"Trend from {start_cycle} to {end_cycle}.\n"
            f"Abnormal = EPS1 < {eps_thr_used:.1f} AND VS1 > {vs_thr_used:.3f}"
        )
        history.append({"role": "assistant", "content": reply})
        return history, state, "", fig, summarize_events(eps_thr_used, vs_thr_used)

    # ============================================================
    # SCATTER MODE
    # ============================================================
    if mode == "scatter":
        fig = plot_scatter(eps_thr_used, vs_thr_used)
        reply = "Scatter (EPS1 vs VS1). High VS1 + low EPS1 = anomaly."
        history.append({"role": "assistant", "content": reply})
        return history, state, "", fig, summarize_events(eps_thr_used, vs_thr_used)

    # ============================================================
    # CAPSULES MODE
    # ============================================================
    if mode == "capsules":
        fig = plot_capsules(eps_thr_used, vs_thr_used, start_cycle, end_cycle)
        reply = "Detected abnormal intervals."
        history.append({"role": "assistant", "content": reply})
        return history, state, "", fig, summarize_events(eps_thr_used, vs_thr_used)

    # ============================================================
    # FAILURE MODE
    # ============================================================
    if mode == "failure":
        prec = df[df["risk"] == 1]
        if prec.empty:
            reply = "No precursor cycles detected."
            fig = plot_help_figure()
        else:
            fig, ax = plt.subplots(figsize=(12, 4))
            ax.plot(df["cycle_id"], df["EPS1_mean"], label="EPS1")
            ax.plot(df["cycle_id"], df["VS1_mean"], label="VS1")
            ax.scatter(prec["cycle_id"], prec["EPS1_mean"], color="red")
            ax.legend()
            reply = f"Detected {len(prec)} precursor cycles."

        history.append({"role": "assistant", "content": reply})
        return history, state, "", fig, ""

    # ============================================================
    # THRESHOLDS MODE
    # ============================================================
    if mode == "thresholds":
        fig = plot_scatter(eps_thr_used, vs_thr_used)
        reply = (
            f"EPS1 abnormal if < {eps_thr_used:.1f}\n"
            f"VS1 abnormal if > {vs_thr_used:.3f}"
        )
        history.append({"role": "assistant", "content": reply})
        return history, state, "", fig, summarize_events(eps_thr_used, vs_thr_used)

    # ============================================================
    # UNKNOWN REQUEST
    # ============================================================
    fig = plot_help_figure()
    reply = (
        "I didn’t understand. Try:\n"
        "- explain spike around cycle 1000\n"
        "- show trend 200 to 500\n"
        "- find anomalies\n"
    )
    history.append({"role": "assistant", "content": reply})
    return history, state, "", fig, ""


# ============================================================
# BUILD GRADIO APP (your UI preserved)
# ============================================================

with gr.Blocks() as demo:
    gr.Markdown("## EPS1 & VS1 Risk Dashboard – Trends, Capsules, Scatter & Chat")

    with gr.Tab("Dashboard"):
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### Controls")

                eps_thr_in = gr.Slider(
                    minimum=float(cycles["EPS1_mean"].min()),
                    maximum=float(cycles["EPS1_mean"].max()),
                    value=float(eps1_thresh_default),
                    step=1.0,
                    label="EPS1_mean threshold"
                )

                vs_thr_in = gr.Slider(
                    minimum=float(cycles["VS1_mean"].min()),
                    maximum=float(cycles["VS1_mean"].max()),
                    value=float(vs1_thresh_default),
                    step=0.01,
                    label="VS1_mean threshold"
                )

                start_cycle_in = gr.Slider(1, max_cycle_id, value=1, step=1, label="Start cycle")
                end_cycle_in = gr.Slider(1, max_cycle_id, value=max_cycle_id, step=1, label="End cycle")

            with gr.Column(scale=2):
                trend_out = gr.Plot(label="EPS1 & VS1 Trends")
                scatter_out = gr.Plot(label="Risk Map")
                events_out = gr.Textbox(label="High-risk events", lines=10)
                tree_out = gr.Plot(label="Decision Tree")

        def update_dashboard(eps_thr, vs_thr, start_cycle, end_cycle):
            start_cycle, end_cycle = clamp_range(start_cycle, end_cycle)
            return (
                plot_trends(eps_thr, vs_thr, start_cycle, end_cycle),
                plot_scatter(eps_thr, vs_thr),
                summarize_events(eps_thr, vs_thr),
                plot_decision_tree()
            )

        demo.load(update_dashboard, [eps_thr_in, vs_thr_in, start_cycle_in, end_cycle_in],
                  [trend_out, scatter_out, events_out, tree_out])

        eps_thr_in.change(update_dashboard, [eps_thr_in, vs_thr_in, start_cycle_in, end_cycle_in],
                          [trend_out, scatter_out, events_out, tree_out])
        vs_thr_in.change(update_dashboard, [eps_thr_in, vs_thr_in, start_cycle_in, end_cycle_in],
                         [trend_out, scatter_out, events_out, tree_out])
        start_cycle_in.change(update_dashboard, [eps_thr_in, vs_thr_in, start_cycle_in, end_cycle_in],
                              [trend_out, scatter_out, events_out, tree_out])
        end_cycle_in.change(update_dashboard, [eps_thr_in, vs_thr_in, start_cycle_in, end_cycle_in],
                            [trend_out, scatter_out, events_out, tree_out])

    # ---------------- CHAT TAB ----------------
    with gr.Tab("Chat"):
        gr.Markdown("### Ask anything about anomalies, failures, vibration spikes, capsules, thresholds, trends.")

        with gr.Row():
            with gr.Column(scale=1):
                chatbot = gr.Chatbot(height=400)
                msg_box = gr.Textbox(label="Your question", lines=2)
                send_btn = gr.Button("Send")
                chat_state = gr.State({"history": [], "last_start": 1, "last_end": max_cycle_id})

            with gr.Column(scale=1):
                chat_plot = gr.Plot(label="Visualization")
                chat_events = gr.Textbox(label="Event summary", lines=12)

        send_btn.click(
            fn=chat_handle,
            inputs=[msg_box, chat_state, eps_thr_in, vs_thr_in],
            outputs=[chatbot, chat_state, msg_box, chat_plot, chat_events]
        )

# ============================================================
# Launch app
# ============================================================
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c8728a2a12c7701534.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
